In [1]:
# default_exp datasets

# Dataset Pages Generation

<br>

### Imports

In [2]:
#exports
import json
import yaml
import numpy as np
import pandas as pd

from frictionless import Package

import os
import shutil
from tqdm import tqdm
from pathlib import Path
from jinja2 import Template

from powerdict import dictionary

In [3]:
from IPython.display import JSON, Markdown

In [4]:
dataset = 'global-power-plant-database'
datapackage_json_fp = f'../data/attribute_sources/{dataset}/datapackage.json'

package = Package(datapackage_json_fp, profile='tabular-data-package')

In [22]:
#exports
resource_to_download_url = lambda resource, package_name, root_path='https://osuked.github.io/Power-Station-Dictionary/attribute_sources': f'{root_path}/{package_name}/{resource["path"]}'

def resource_to_title(resource):
    if 'title' in resource.keys():
        title = resource['title']
    else:
        title = resource['name']
    
    return title

def resource_to_description(resource):
    if 'description' in resource.keys():
        description = resource['description']
    else:
        description = False
    
    return description

def package_to_homepage(package):
    if 'homepage' in package.keys():
        homepage = package['homepage']
    else:
        homepage = False
    
    return homepage

def resource_to_fields_table_str(resource):
    df_fields = pd.DataFrame(resource['schema']['fields'])

    df_fields = df_fields.rename(columns={'name': 'column'})
    df_fields.columns = df_fields.columns.str.capitalize()

    md_str = df_fields.to_markdown(index=False)
    
    return md_str

def extract_resource_elements(package):
    resource_elements = [
        {
            'title': resource_to_title(resource),
            'download_url': resource_to_download_url(resource, package['name']),
            'description': resource_to_description(resource),
            'fields': resource_to_fields_table_str(resource)
        }
        for resource 
        in package.resources
    ]

    return resource_elements

def extract_package_elements(package):
    package_elements = {
        'title': resource_to_title(package),
        'homepage': package_to_homepage(package),
        'description': resource_to_description(package),
        'metadata': dictionary.construct_metadata_table_str(package),
        'resource_elements': extract_resource_elements(package)
    }
    
    return package_elements

In [6]:
package_elements = extract_package_elements(package)

JSON(package_elements)

<IPython.core.display.JSON object>

In [7]:
template_fp = '../templates/dataset_page.md'
save_fp = f'../docs/datasets/{package["name"]}.md'

dictionary.populate_and_save_template(template_fp, save_fp, package_elements=package_elements)

<br>

### Iterating Over All Datasets

In [8]:
#exports
def identify_valid_datasets(datasets_dir):
    dataset_dirs = [
        elem 
        for elem 
        in os.listdir(datasets_dir) 
        if elem != '.ipynb_checkpoints'
    ]

    valid_datasets = [
        dataset_dir
        for dataset_dir 
        in dataset_dirs
        if 'datapackage.json' in os.listdir(f'{datasets_dir}/{dataset_dir}')
    ]
    
    return valid_datasets

In [9]:
datasets_dir = '../data/attribute_sources'

valid_datasets = identify_valid_datasets(datasets_dir)

valid_datasets[:5]

['annual-output',
 'bmu-fuel-types',
 'capture-prices',
 'cfd-contract-portfolio-status',
 'cfd-strike-prices']

In [12]:
#exports
def update_mkdocs_config(
    valid_datasets: list,
    dataset_to_name: dict,
    mkdocs_config_fp: str
):
    with open(mkdocs_config_fp) as fp:
         mkdocs_config = yaml.safe_load(fp)

    nav_datasets_idx = [idx for idx, nav_item in enumerate(mkdocs_config['nav']) if 'Datasets' in nav_item.keys()][0]
    mkdocs_config['nav'][nav_datasets_idx]['Datasets'] = {f'{dataset_to_name[dataset]}': f'datasets/{dataset}.md' for dataset in valid_datasets}

    with open(mkdocs_config_fp, 'w') as fp:
        yaml.dump(mkdocs_config, fp)
        
    return

def populate_dataset_pages(
    datasets_dir: str,
    dataset_template_fp: str,
    mkdocs_config_fp: str='../mkdocs.yml'
):
    dataset_to_name = {}
    valid_datasets = identify_valid_datasets(datasets_dir)

    for dataset in tqdm(valid_datasets):
        datapackage_json_fp = f'../data/attribute_sources/{dataset}/datapackage.json'
        package = Package(datapackage_json_fp, profile='tabular-data-package')

        package_elements = extract_package_elements(package)
        dataset_to_name[dataset] = package_elements['title'].strip()
        save_fp = f'../docs/datasets/{package["name"]}.md'
        dictionary.populate_and_save_template(dataset_template_fp, save_fp, package_elements=package_elements)
        
    update_mkdocs_config(valid_datasets, dataset_to_name, mkdocs_config_fp)
    
    return 

In [13]:
mkdocs_config_fp = '../mkdocs.yml'
dataset_template_fp = '../templates/dataset_page.md'

populate_dataset_pages(datasets_dir, dataset_template_fp, mkdocs_config_fp)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 62.49it/s]


In [14]:
#exports
def move_attribute_source_data_to_docs(
    src_path: str='../data/attribute_sources/',
    trg_path: str='../docs/attribute_sources/'
):
    shutil.rmtree(trg_path)
    shutil.copytree(src_path, trg_path)
    
    return

In [15]:
move_attribute_source_data_to_docs()

In [23]:
#hide
from nbdev.export import *
notebook2script()

Converted 00-documentation.ipynb.
Converted 01-dictionary-page.ipynb.
Converted 02-attribute extraction.ipynb.
Converted 03-page-population.ipynb.
Converted 04-cli.ipynb.
Converted 05-carbon-intensity.ipynb.
Converted 06-cfd-capture-price-comparison.ipynb.
Converted 07-dataset-pages.ipynb.
